In [ ]:
# Load packages
import pandas as pd
from datetime import datetime
from zillow_scraper import zillow_scraper, make_frame_sales, make_frame_rentals, collect_real_estate_data

locations = ["manhattan-ny"]
property_types = ["rent", "sale"]

In [ ]:
# basic version
collect_real_estate_data(locations=locations, property_types=property_types)

In [ ]:
# OR....if you want to toggle with the settings...this is what is going on in collect_real_estate_data()...
today = datetime.now()
todays_date = today.strftime("%Y_%m_%d")
todays_date

# get rental and sale data for manhattan

for location in locations:
    for property_type in property_types:
        data_dict = zillow_scraper(city=location, property_type=property_type, time_between_scrapes=120, testing=False, min_price=0)

        data = data_dict["data_list"]
        min_price = data_dict["min_price"]
        num_listings = data_dict["num_listings"]
        print(location, property_type, min_price, num_listings)
        df = pd.DataFrame()

        if property_type == "sale":
            # extract into a table
            df = make_frame_sales(df, data)

            #drop cols
            df = df.drop('hdpData', 1) #remove this line to see a whole bunch of other random cols, in dict format

            #drop dupes
            df = df.drop_duplicates(subset='zpid', keep="last")

            #filters
            df['zestimate'] = df['zestimate'].fillna(0)
            df['best_deal'] = df['unformattedPrice'] - df['zestimate']
        else:
            df = make_frame_rentals(df, data)
        
        df.to_csv("{location}_{property_type}_{date}.csv".format(location=location, property_type=property_type, date=todays_date))